In [1]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from numpy import linspace, array, logspace, sin, cos, pi, arange, sqrt, arctan2, arccos
import re

import astropy.io.fits as pyfits



In [62]:
#Grab the existing table columns we want 
cat = Table.read('../3D_CMZ/LBV_plots_stuff/CMZ_cloud_catalogue_data.tex')
absorp_tab = Table.read('../3D_CMZ/LBV_plots_stuff/absorption_table_merged.tex')

absorp_NF_tab = pd.read_csv('../3D_CMZ/LBV_plots_stuff/paper3_agreement_catalogue.txt',sep=',',
                            usecols=[0,1,2,3,4], names=['i', 'l', 'b', 'v', 'NF'],
                            header=None, index_col=False)

absorp_sigma_tab = pd.read_csv('./3dcmz_sigmas_masked.csv',sep=',',
                            usecols=[0,1], names=['ID','sigma_frac_abs'],
                            header=0, index_col=False)

absorp_mass_tab= pd.read_csv('./paper3-cloud-catalogue.csv',sep=',',
                            usecols=[0,6], names=['#','Mass_msun'],
                            header=0, index_col=False)


cat_index = cat['index']
cloud_list = cat['cloud_name']
l = cat['l']
b = cat['b']
v = cat['v']
sigma = cat['sigma']
rad = cat['rad']
NF_decision = cat['NF_decision']

lb_pixel_mask = cat['lb_pixel_mask']

corr_coeff = cat['r_p_max']
flux_diff = cat['flux_diff_max']
flux_ratio = cat['flux_ratio_max']

absorp_value = absorp_tab['fraction_value_peak_mask']
absorp_index = absorp_tab['leaf_id']
absorp_mass = absorp_mass_tab['Mass_msun']




absorp_tab[20]

leaf_id,minimum_h2co,median_cband_weighted,fraction_value_weighted,median_cband_peak_mask,fraction_value_peak_mask,velocity,sigma
str3,float64,float64,float64,float64,float64,int64,int64
17a,-42.7,7.2,0.17,11.9,0.28,18,16


In [66]:
absorp_mass_tab

,#,Mass_msun
0,1,60000.0
1,2,290000.0
2,3,21000.0
3,4,81000.0
4,5,12000.0
5,6,87000.0
6,7,14000.0
7,8,11000.0
8,9,310000.0
9,10,62000.0


In [67]:
absorp_NF_tab['NF'][np.where(absorp_NF_tab['NF']=='Far')[0]] = 'F'
absorp_NF_tab['NF'][np.where(absorp_NF_tab['NF']=='Near')[0]] = 'N'

NF_absorp = absorp_NF_tab['NF'].values

/var/folders/4b/_cshhy012yj8k_187jxt881r0000gn/T/ipykernel_2602/3162031848.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  absorp_NF_tab['NF'][np.where(absorp_NF_tab['NF']=='Far')[0]] = 'F'
/var/folders/4b/_cshhy012yj8k_187jxt881r0000gn/

In [68]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [83]:
### Save mass array for each index, including duplicated indices
mass_arr = []
for i in cat_index:
    indnum = natural_keys(i)[1]
    match_massind = np.where(indnum == absorp_mass_tab['#'].values)[0][0]
    mass_arr.append(float(absorp_mass_tab['Mass_msun'].values[match_massind]))
mass_arr = [f'{num:.1e}' for num in mass_arr]

In [70]:
absorp_index_digits = [re.findall("\d+", i)[0] for i in absorp_index]
absorp_tab_ids = []
for j in absorp_sigma_tab['ID']:
    spl_id = j.split('_')[0]
    spl_v = j.split('_')[1].replace("v", "")

    search_cond = (np.array(absorp_index_digits) == str(spl_id)) & (absorp_tab['velocity'] == int(spl_v))
    absorp_tab_ids.append(str(absorp_index[search_cond][0]))  

absorp_sigma_tab['leaf_id'] = absorp_tab_ids
#absorp_tab_ids.sort(key=natural_keys)
#absorp_tab['IDs'] = absorp_tab_ids


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/var/folders/4b/_cshhy012yj8k_187jxt881r0000gn/T/ipykernel_2602/59385925.py:1: SyntaxWarning: invalid escape sequence '\d'
  absorp_index_digits = [re.findall("\d+", i)[0] for i in absorp_index]


In [71]:
###just need this file for the multi-comp clouds
large_cat = Table.read('../3D_CMZ/LBV_plots_stuff/CMZ_cloud_catalogue_data_dup.csv')
large_cat_index = large_cat['index']
large_cat_v = large_cat['v']

In [72]:
### For the stddevs, we need to find them for both the single 
### and multi-component clouds separately 
### then combine into the table 

#Find single v component clouds
single_v = {x for x in list(cloud_list) if list(cloud_list).count(x) == 1}


sing_index =[]
for i in range(len(cloud_list)):
    if cloud_list[i] in single_v:
        sing_index.append(i)
        

#find indexes with multiple velocities 
dup = {x for x in list(cloud_list) if list(cloud_list).count(x) > 1}

dup_index =[]
for i in range(len(cloud_list)):
    if cloud_list[i] in dup:
        dup_index.append(i)
        
id_v_list = []
for i in range(len(large_cat_index)):
    id_v_list.append('{}_{}'.format(large_cat_index[i], large_cat_v[i]))

In [73]:
flux_ratio_std = []
flux_diff_std = []

for i in range(len(cat_index)):
    if cat_index[i] in cat_index[sing_index]:
        cloud = cloud_list[i]
        
        cloud_flux_diff = pyfits.open('../3D_CMZ/Cloud_masks/{}/{}_flux_diff.fits'.format(cloud, cloud))[0].data
        fdiff_std = np.abs(np.nanstd(cloud_flux_diff))
        flux_diff_std.append('{:.2f}'.format(np.round(fdiff_std,2)))

        cloud_flux_ratio = pyfits.open('../3D_CMZ/Cloud_masks/{}/{}_flux_ratio.fits'.format(cloud, cloud))[0].data
        fratio_std = np.abs(np.nanstd(cloud_flux_ratio))
        flux_ratio_std.append('{:.2f}'.format(np.round(fratio_std,2)))
        
    if cat_index[i] in cat_index[dup_index]:
        cloud_flux_diff = pyfits.open('../3D_CMZ/Sub_masks/{}/max_submask/{}_{}_flux_diff_maxcutout_isolated.fits'.format(large_cat_index[i], large_cat_index[i],large_cat_v[i] ))[0].data
        fdiff_std = np.abs(np.nanstd(cloud_flux_diff))
        flux_diff_std.append('{:.2f}'.format(np.round(fdiff_std,2)))

        cloud_flux_ratio = pyfits.open('../3D_CMZ/Sub_masks/{}/max_submask/{}_{}_flux_ratio_maxcutout_isolated.fits'.format(large_cat_index[i], large_cat_index[i],large_cat_v[i] ))[0].data
        fratio_std = np.abs(np.nanstd(cloud_flux_ratio))
        flux_ratio_std.append('{:.2f}'.format(np.round(fratio_std,2)))

/opt/anaconda3/envs/python3_13/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [84]:
### absorption table only has the ids with measurement available

#create nan array with size equal to 'index'
absorp_arr = []
absorp_NF = []
absorp_sig_arr = []

#fill in where indices match. Make nan where no match.
for i in cat_index:
    if i in absorp_index:
        if i =='16a': #16a is an empty mask. replace with nan
            absorp_arr.append(np.nan)
        else: 
            absorp_arr.append(absorp_value[np.where(absorp_index==str(i))[0][0]])
    else: 
        absorp_arr.append(np.nan)
        
for i in cat_index:
    if i in absorp_NF_tab['i'].values:
        if i =='16a': #16a is an empty mask. replace with nan
            absorp_NF.append(np.nan)
        else: 
            absorp_NF.append(absorp_NF_tab['NF'][np.where(absorp_NF_tab['i']==str(i))[0][0]])
    else: 
        absorp_NF.append(np.nan)

#fill in where indices match. Make nan where no match.
for i in cat_index:
    if i in absorp_tab_ids:
        if i =='16a': #16a is an empty mask. replace with nan
            absorp_sig_arr.append(np.nan)
        else: 
            absorp_sig_arr.append(float(absorp_sigma_tab['sigma_frac_abs'][np.where(absorp_sigma_tab['leaf_id'] == str(i))[0][0]]))
    else: 
        absorp_sig_arr.append(np.nan)

In [87]:
### Construct new table 

synth_table = Table()
synth_table['leaf_id'] = cat_index
synth_table['cloud_name'] = cat['cloud_name']
synth_table['l'] = l
synth_table['b'] = b
synth_table['v'] = v
synth_table['sigma'] = sigma
synth_table['rad'] = rad
synth_table['mass_msun'] = mass_arr
synth_table['corr_coeff'] = corr_coeff.round(2)
synth_table['flux_diff'] = flux_diff.round(2)
synth_table['flux_diff_stdv'] = flux_diff_std
synth_table['flux_ratio'] = flux_ratio.round(2)
synth_table['flux_ratio_stdv'] = flux_ratio_std
synth_table['absorp_value'] = absorp_arr
synth_table['absorp_sigma'] = absorp_sig_arr
synth_table['NF_decision'] = NF_decision
synth_table['absorp_NF'] = absorp_NF
synth_table['lb_pixel_mask'] = lb_pixel_mask

synth_table


leaf_id,cloud_name,l,b,v,sigma,rad,mass_msun,corr_coeff,flux_diff,flux_diff_stdv,flux_ratio,flux_ratio_stdv,absorp_value,absorp_sigma,NF_decision,absorp_NF,lb_pixel_mask
str3,str14,float64,float64,int64,int64,float64,str7,float64,float64,str6,float64,str4,float64,float64,str3,str32,int64
1,G359.475-0.044,-0.525,-0.044,-102,6,4.3,6.0e+04,0.26,-20.97,59.09,0.37,0.20,nan,nan,LN,nan,1
2,G359.508-0.135,-0.492,-0.135,-56,9,8.7,2.9e+05,0.38,-73.25,56.22,0.46,0.19,nan,nan,N,nan,1
3,G359.561-0.001,-0.439,-0.001,-90,8,2.7,2.1e+04,0.22,-59.31,23.73,0.38,0.09,2.15,0.06,U,F,1
4a,G359.595-0.223,-0.405,-0.223,-27,13,4.6,8.1e+04,0.43,-79.52,73.95,0.37,0.30,nan,nan,N,nan,1
4b,G359.595-0.223,-0.405,-0.223,-20,8,4.6,8.1e+04,0.26,-78.19,101.97,0.36,0.42,nan,nan,LN,nan,0
5,G359.608+0.018,-0.392,0.018,-78,5,2.0,1.2e+04,0.18,-79.45,23.27,0.38,0.11,0.52,0.07,LN,N,1
6a,G359.688-0.132,-0.312,-0.132,-29,5,5.6,8.7e+04,0.62,-52.41,25.94,0.4,0.10,3.3,0.11,LN,F,0
6b,G359.688-0.132,-0.312,-0.132,-21,12,5.6,8.7e+04,0.28,-61.84,29.84,0.41,0.09,2.28,0.05,U,F,1
7a,G359.701+0.032,-0.299,0.032,-73,11,2.4,1.4e+04,-0.54,-96.86,32.34,0.46,0.12,0.36,0.05,LN,N,1


In [88]:
synth_table.write("/Users/danilipman/Documents/Research/UConn/CMZ_SYNTH/synth_table.tex", overwrite=True)

    


In [44]:
print(absorp_index[22], absorp_value[22])

17c 1.63


In [45]:
synth_table[16]

leaf_id,cloud_name,l,b,v,sigma,rad,corr_coeff,flux_diff,flux_diff_stdv,flux_ratio,flux_ratio_stdv,absorp_value,NF_decision,absorp_NF,lb_pixel_mask
str3,str14,float64,float64,int64,int64,float64,float64,float64,str6,float64,str4,float64,str3,str32,int64
11b,G0.014-0.016,0.014,-0.016,45,9,3.4,-0.35,-177.87,101.24,0.92,0.19,2.8,F,nan,0
